In [1]:
from langchain_community.utilities import SearxSearchWrapper
from pydantic import Field
import requests

from beeai_framework.backend.chat import ChatModel, ChatModelOutput, ChatModelStructureOutput
from beeai_framework.backend.message import UserMessage
from beeai_framework.template import PromptTemplate, PromptTemplateInput
import traceback
from pydantic import BaseModel, ValidationError
from beeai_framework.workflows.workflow import Workflow, WorkflowError
from pydantic import InstanceOf
from beeai_framework.backend.message import AssistantMessage, SystemMessage
from beeai_framework.memory.unconstrained_memory import UnconstrainedMemory
from beeai_framework.agents.react.agent import ReActAgent
from beeai_framework.tools import Tool, tool
from dotenv import load_dotenv
import os
from langchain_community.tools import TavilySearchResults  # Import LangChain Tavily tool
from typing import Any

from beeai_framework.agents.react.agent import ReActAgent
from beeai_framework.agents.react.types import ReActAgentRunOutput
from beeai_framework.backend.chat import ChatModel
from beeai_framework.emitter.emitter import Emitter, EventMeta
from beeai_framework.emitter.types import EmitterOptions
from beeai_framework.memory.unconstrained_memory import UnconstrainedMemory


In [2]:
os.getcwd()

'/Users/sandi/beeai-framework/python/examples/notebooks'

In [3]:
#Load Tavily API Key

# Check if the file exists
env_path = os.path.join(os.getcwd(), ".env")
print(f"Does .env exist? {'Yes' if os.path.exists(env_path) else 'No'}")

load_dotenv(dotenv_path=env_path)

tavily_api_key = os.getenv("TAVILY_API_KEY")

Does .env exist? Yes


In [4]:
model = ChatModel.from_name("ollama:granite3.1-dense:2b")

In [5]:
# Workflow Overall State
class SearchAgentState(BaseModel):
    question: str
    websearch_query: str | None = None
    search_results: str | None = None
    answer: str | None = None
    memory: InstanceOf[UnconstrainedMemory]

In [6]:
#structured output for the web search terms used by generate_web_search_terms function
class WebSearchQuery(BaseModel):
    search_query: str = Field(description="The web search query.")

# class WebSearchResults(BaseModel):
#     results: str = Field(description="The web search results from the tool call.")

#structured input for creating search terms step
class QuestionInput(BaseModel):
    question: str

#prompt template to create search terms
search_query_template = PromptTemplate(
    PromptTemplateInput(
        schema=QuestionInput,
        template="""Convert the following question into a concise, effective web search query using keywords and operators for accuracy.
Question: {{question}}""",
    )
)

class GeneratedOutput(BaseModel):
    search_answer: str = Field(description="The output from the agent.")

In [7]:
#Functions for the Re-ACT Agent Process

# Observe the agent
async def observer(emitter: Emitter) -> None:
    emitter.on("*.*", process_agent_events, EmitterOptions(match_nested=True))

#process agent events
async def process_agent_events(event_data: dict[str, Any], event_meta: EventMeta) -> None:
    """Process agent events and log appropriately"""

    if event_meta.name == "error":
        print("Agent 🤖 : ", event_data["error"])
    elif event_meta.name == "retry":
        print("Agent 🤖 : ", "retrying the action...")
    elif event_meta.name == "update":
        print(f"Agent({event_data['update']['key']}) 🤖 : ", event_data["update"]["parsedValue"])

In [8]:
@tool
def tavily_search_tool(query: str) -> str:
    """
    Perform a web search for the latest and most relevant information available online.
    
    Args:
        query (str): The search query to look up on the web which comes from the websearch_query.
    
    Returns:
        str: The top search results based on the Tavily API.
    """
    if not isinstance(query, str) or not query.strip():
        raise ValueError("Invalid query input. Expected a non-empty string.")

    tavily_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=False)

    searchresults = tavily_tool.invoke({"query": query})
    searchresults = str(searchresults)
    return searchresults

In [9]:
#STEP IN WORKFLOW
async def generate_web_search_terms(state: SearchAgentState) -> str:
    print("Step: ", "generate_web_search_terms")
    # Generate a search query
    prompt = search_query_template.render(QuestionInput(question=state.question))
    response: ChatModelStructureOutput = await model.create_structure(
        schema=WebSearchQuery, messages=[UserMessage(prompt)]
    )
    #add the assistant response to websearch_query in state 
    state.websearch_query = response.object["search_query"]
    return "generate_answer"

In [10]:
# prompt = search_query_template.render(QuestionInput(question=state.question))
# response: ChatModelStructureOutput = await model.create_structure(
#         schema=WebSearchQuery, messages=[UserMessage(prompt)]
#     )


#STEP IN WORKFLOW
async def generate_answer(state: SearchAgentState) -> str:
    print("Step: ", "generate_answer")
    
    # Ensure we are passing the correct search query
    if not isinstance(state.websearch_query, str) or not state.websearch_query.strip():
        raise ValueError("websearch_query is invalid. Ensure it is a non-empty string.")
    

    #create the agent
    agent = ReActAgent(llm=model, tools=[tavily_search_tool], memory=UnconstrainedMemory())
    
    # Debugging
    print(f"Running agent with query: {state.websearch_query}")
    print (type(state.websearch_query))
    #run the agent

    result: ReActAgentRunOutput = await agent.run(state.websearch_query).observe(observer)

    # Store answer in state. THIS IS THE ISSUE!!
   # state.answer = result.result.content

    # Add response to memory
    await state.memory.add(AssistantMessage(content=state.answer))
    
    return Workflow.END


In [ ]:
help(ReActAgentRunOutput)

In [11]:
#add an inital system prompt to memory so that it is not empty when the process kicks off

memory = UnconstrainedMemory()
await memory.add(SystemMessage(content="You are an AI assistant that helps people understand their medical bills. You have access to a web search tool to help you gather information about the codes and provide the user with accurate, up-to-date information."))

In [12]:
try:
    search_workflow = Workflow(SearchAgentState)
    
    # Define workflow steps
    search_workflow.add_step("generate_web_search_terms", generate_web_search_terms)
    search_workflow.add_step("generate_answer", generate_answer)

    while True:
        user_input = input("User (type 'exit' to stop): ")
        if user_input.lower() == "exit":
            break

        # Add user message to memory
        await memory.add(UserMessage(content=user_input))

        # Run workflow with memory
        response = await search_workflow.run(SearchAgentState(question=user_input, memory=memory))
        # Print assistant response
        print("Assistant:", response.state.answer)

except WorkflowError:
    traceback.print_exc()
except ValidationError:
    traceback.print_exc()

Step:  generate_web_search_terms
Step:  generate_answer
Running agent with query: Code 90658 billing explanation
<class 'str'>
Agent 🤖 :  The generated output does not adhere to the schema.
Transition from 'thought' to 'tool_input' does not exist!
Agent 🤖 :  The generated output does not adhere to the schema.
Transition from 'thought' to 'tool_input' does not exist!
Agent 🤖 :  The generated output does not adhere to the schema.
Transition from 'thought' to 'tool_input' does not exist!
Agent 🤖 :  retrying the action...
Agent(thought) 🤖 :  I need to find out about Code 90658 billing explanation using the Tavily search tool.

Agent(tool_name) 🤖 :  tavily_search_tool
Agent(tool_input) 🤖 :  {'query': 'Code 90658 billing explanation'}
Agent(tool_output) 🤖 :  [{'title': 'CPT Code 90658: What It Is, Modifiers, Reimbursement - MD Clarity', 'url': 'https://www.mdclarity.com/cpt-code/90658?10534572_page=31', 'content': 'CPT code 90658 is used to describe the administration of the trivalent influe